# Master Universitario en Lógica, Computación e Inteligencia Artificial
## Aprendizaje Automático
**Autor:** Arturo Pérez Sánchez &nbsp;&nbsp;&nbsp;
## Cuestionario 4: implementación de árboles de decisión

## Contenido

1. <a href="#ej1">Reproducción los pasos que se han visto en clase sobre el conjunto iris</a> <br>
    1.a - <a href="#ej1-a"> Carga del fichero y preprocesamiento </a> <br>
    1.b - <a href="#ej1-b"> examen exploratorio </a> <br>
    1.c - <a href="#ej1-c"> División del dataset y creación del árbol </a> <br>
    1.d - <a href="#ej1-d"> Representación gráfica del árbol </a> <br>
    1.e - <a href="#ej1-e"> Medidas de rendimiento sobre conjuntos de entrenamiento, prueba y total </a> <br>

2. <a href="#ej2"> Análisis adicional </a> <br>
    2.a -  <a href="#ej2-a"> Validación cruzada </a> <br>
    2.b -  <a href="#ej2-b"> Optimización de parámetros con SearchGrid </a> <br>
    2.c -  <a href="#ej2-c"> Simplificación del modelo </a> <br>
    2.d -  <a href="#ej2-d"> Árbol ID3 </a> <br>
    2.e -  <a href="#ej2-e"> Matriz de confusión </a> <br>
    2.f -  <a href="#ej2-f"> Otras métricas </a> <br>
    

Para el desarrollo de este proyecto había que utilizar un dataset del repositorio de la Universidad de California, por lo que tras probar con varios, me he decidido por utilizar el dataset <a href="https://archive.ics.uci.edu/ml/datasets/Early+stage+diabetes+risk+prediction+dataset."> Early stage diabetes risk prediction dataset </a>. Con estos datos intentaremos clasificar si un paciente tiene o no diabetes en función de varios parámetros<br><br>
Este dataset consta de 520 filas (instancias) y 17 columnas (atributos). <br><br>

# 1 Reproducción los pasos que se han visto en clase sobre el conjunto iris <a name="ej1"> </a>

Lo primero que tenemos que hacer antes de comenzar el análisis exploratorio es importar todas las librerias necesarias:

In [ ]:
from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_text
from sklearn.tree import export_graphviz
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import sklearn.metrics
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.metrics as skm
import graphviz

Una vez importadas todas las dependencias necesarias podemos comenzar con el primer punto: 

### 1.a) Carga del fichero y preprocesamiento <a name="ej1-a"> </a>

Como el repositorio de la universidad de California nos ofrece los datos ya en formato .csv podemos leer directamente los datos desde la url:

In [ ]:
dataset = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00529/diabetes_data_upload.csv')

Una vez cargado el dataset podemos imprimirlo en pantalla para ver los atributos:

In [ ]:
dataset

Como podemos ver, todos los atributos a excepción de la edad son de tipo categórico, pero el algoritmo que vamos a utilizar necesita valores numéricos, por ello lo primero que vamos a hacer es manipular los datos para convertirlos a valores enteros, como los atributos son binarios, sustituiremos el valor "Yes" por 1 y "No" por 0.

Cabe destacar que la variable que queremos predecir es "class" que tambien convertiremos a valores de tipo entero (1 para positivo y 0 para negativo)

In [ ]:
dataset['Gender'] = dataset['Gender'].map({'Male':1 ,'Female':0})
dataset['Polyuria'] = dataset['Polyuria'].map({'Yes':1 ,'No':0})
dataset['Polydipsia'] = dataset['Polydipsia'].map({'Yes':1 ,'No':0})
dataset['sudden weight loss'] = dataset['sudden weight loss'].map({'Yes':1 ,'No':0})
dataset['weakness'] = dataset['weakness'].map({'Yes':1 ,'No':0})
dataset['Polyphagia'] = dataset['Polyphagia'].map({'Yes':1 ,'No':0})
dataset['Genital thrush'] = dataset['Genital thrush'].map({'Yes':1 ,'No':0})
dataset['visual blurring'] = dataset['visual blurring'].map({'Yes':1 ,'No':0})
dataset['Itching'] = dataset['Itching'].map({'Yes':1 ,'No':0})
dataset['Irritability'] = dataset['Irritability'].map({'Yes':1 ,'No':0})
dataset['delayed healing'] = dataset['delayed healing'].map({'Yes':1 ,'No':0})
dataset['partial paresis'] = dataset['partial paresis'].map({'Yes':1 ,'No':0})
dataset['muscle stiffness'] = dataset['muscle stiffness'].map({'Yes':1 ,'No':0})
dataset['Alopecia'] = dataset['Alopecia'].map({'Yes':1 ,'No':0})
dataset['Obesity'] = dataset['Obesity'].map({'Yes':1 ,'No':0})

# Atributo que queremos predecir
dataset['class'] = dataset['class'].map({'Positive':1 ,'Negative':0})

a continuación separamos del dataset la clase que queremos clasificar de las demás y las guardamos en una variable separada (y_data)

In [ ]:
y_data = dataset['class'].to_numpy()
dataset = dataset.drop('class', axis=1)
X_data = dataset.to_numpy()

### 1.b) Examen exploratorio <a name="ej1-b"> </a>

Ahora vamos a realizar un pequeño análisis exploratorio del dataset, comenzamos representando la _scatter matrix_

In [ ]:
sp = pd.plotting.scatter_matrix(dataset, c=y_data, figsize=(15, 15), marker='o',
                                 hist_kwds={'bins': 20}, s=60, alpha=.8)

Como la mayoria de atributos son binarios, es normal que los puntos aparezcan siempre en las esquinas, además podemos observar que el único atributo que no es categórico es el primero (la edad).<br>
A continuación representaremos un mapa de calor con los coeficientes de **correlación de Pearson**

In [ ]:
plt.figure(figsize=(15,15))
plt.title('Coeficiente de Correlación de Pearson entre los atributos', y=1.05, size=15)
sns.heatmap(dataset.astype(float).corr(),linewidths=0.1,vmax=1.0, 
            square=True, cmap='viridis', linecolor='white', annot=True)

### 1.c) División del dataset y creación del árbol <a name="ej1-c"> </a>

Primero dividiremos nuestro dataset en conjunto de test y de entrenamiento, concretamente repartiremos un 70% de entrenamiento y un 30% de test, además, crearemos una semilla para los datos no varíen con cada ejecución

In [ ]:
seed = 5382
X_train, X_test, y_train, y_test = train_test_split(X_data,y_data,test_size = 0.30, random_state=seed)

Una vez separados los datos ya podemos crear el árbol de decisión y entrenarlo con los datos de entrenamiento

In [ ]:
modelo = DecisionTreeClassifier(random_state=seed)
modelo.fit(X_train,y_train)

### 1.d) Representación gráfica del árbol <a name="ej1-d"> </a>

Una vez hemos entrenado el árbol podemos utilizar la ibrería graphviz para representar gráficamente en árbol:

In [ ]:
export_graphviz(modelo, out_file='salida.dot', feature_names=dataset.columns, rounded=True, filled=True)
with open("salida.dot") as f:
    grafo = f.read()
graphviz.Source(grafo)

Como puede verse al representar el árbol, hay muchos nodos que se crean para clasificar muy pocos datos, esto podría ser un posible caso de overfitting, en el siguiente apartado estudiaremos el rendimiento con los datos de tests que habiamos separado inicialmente

### 1.d) Medidas de rendimiento sobre conjuntos de entrenamiento, prueba y total<a name="ej1-e"> </a>

Para evaluar el rendimiento del árbol generado podemos utilizar la medida de rendimiento _score_ que trae sklearn implementada, además guardaremos los resultados en un dataframe para compararlo con otros modelos que desarrollaremos en el ejercicio 2

In [ ]:
RESULTADOS = pd.DataFrame(columns=['Train_performance', 'Test_performance','Total_performance'])

RESULTADOS.loc['modelo'] = [modelo.score(X_train,y_train), modelo.score(X_test,y_test),  modelo.score(X_data,y_data)]
RESULTADOS

# 2 Análisis adicional <a name="ej2"> </a>
### 2.a) Validación cruzada <a name="ej2-a"> </a>
Lo primero que vamos a realizar sobre el modelo anterior es cambiar la forma de evaluarlo, por lo que en lugar de dividir el conjunto de datos en entrenamiento y test, utilizaremos validación cruzada (con 25 "carpetas") con estratificación:

In [ ]:
cross_validation = StratifiedKFold(n_splits = 25, shuffle = True, random_state =seed)
scores = cross_val_score(modelo, X_data, y_data, scoring = "accuracy", cv = cross_validation)

print( "Exactitud (accuracy): %0.5f (+/- %0.5f)" % (scores.mean(), scores.std() * 2) )

RESULTADOS.loc['modelo + CV'] = ['N/A', 'N/A',  scores.mean()]
RESULTADOS

### 2.b) Optimización de parámetros con SearchGrid <a name="ej2-b"> </a>

Otro de las cosas que podemos hacer es buscar los mejores valores para varios parámetros a la vez utilizando [SearchGrid](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).

In [ ]:
parameters = {
    'criterion':['gini', 'entropy'],
    'max_depth':[None, 5, 10, 15],
    'max_features':['auto', 'sqrt', 'log2'],
    'max_leaf_nodes':[None, 25, 50],
    'min_impurity_decrease':[0.0, 0.001, 0.01],
    'min_samples_leaf':[8, 10, 12, 14],
    'min_samples_split':[2, 15, 30, 45]
}

modelo_opt = GridSearchCV(DecisionTreeClassifier(random_state=seed), parameters, cv = cross_validation, n_jobs=2, scoring='accuracy')
modelo_opt.fit(X_data, y_data)
print('Mejores hiperparámetros encontrados:', modelo_opt.best_params_)

In [ ]:
RESULTADOS.loc['modelo_opt'] = [modelo_opt.score(X_train,y_train), modelo_opt.score(X_test,y_test),  modelo_opt.score(X_data,y_data)]
RESULTADOS

Aunque parezca que utilizando los atributos óptimos se obtiene un resultado peor esto no es cierto, esto ocurre porque el primer modelo se están evaluando perfectamente todos los atributos del set de entrenamiento, lo que hace que el rendimiento sobre el dataset total sea mejor, sin embargo, en el rendimiento respecto al conjunto de test poddemos observar como el modelo con la optimización de parámetros es mejor.

### 2.c) Simplificación de modelo <a name="ej2-c"> </a>
Como hemos visto, el modelo se ajusta bastante bien al conjunto de datos, sin embargo, cuando representamos el árbol este es bastante extenso y dificil de comprender, por lo que vamos a estudiar si podemos disminuir la profundidad del árbol sin que afecte en exceso al rendimiento

In [ ]:
max_depth = 17

res = [0, 0]

for i in range(1, max_depth):
    tree_aux = DecisionTreeClassifier(
    criterion = 'entropy',
    max_depth = i, 
    max_features = 'auto',
    max_leaf_nodes = None, 
    min_impurity_decrease = 0.0, 
    min_samples_leaf = 8,
    min_samples_split = 2)
    scores = cross_val_score(tree_aux, X_data, y_data, scoring = "accuracy", cv = cross_validation)
    res.append(scores.mean())
plt.plot(res)

Aqui podemos ver que a partir de profundidad 4 ya no se nota una mejora significativa, por lo que podemos representar este árbol que al tener menor profundidad se entenderá mejor 

In [ ]:
modelo_depth5 = DecisionTreeClassifier(criterion='entropy', max_depth=4, max_features='auto',
    max_leaf_nodes=None, min_impurity_decrease=0.0, min_samples_leaf=8, min_samples_split=2)

modelo_depth5.fit(X_data, y_data)

export_graphviz(modelo_depth5, out_file='salida.dot', feature_names=dataset.columns, rounded=True, filled=True)
with open("salida.dot") as f:
    grafo = f.read()
graphviz.Source(grafo)

### 2.d) Árbol ID3<a name="ej2-d"> </a>

Todos los modelos que hemos desarrollado han sido utilizando el mismo algoritmo de aprendizaje de árboles de decisión que está incluido en *scikit_learn*, que es [CART](https://books.google.es/books/about/Classification_and_regression_trees.html?id=uxPvAAAAMAAJ&redir_esc=y), sin embargo, podemos utilizar otras librerias para utilizar otros algoritmos como los vistos en clase.

In [ ]:
# !pip install decision-tree-id3
import six
import sys
sys.modules['sklearn.externals.six'] = six
from id3 import Id3Estimator

modeloID3 = Id3Estimator()
modeloID3.fit(X_train,y_train)

In [ ]:
prediction0 = modeloID3.predict(X_train)
prediction1 = modeloID3.predict(X_test)
prediction2 = modeloID3.predict(X_data)

score0 = sum(x1==x2 for x1,x2 in zip(prediction0,y_train))/len(y_train)
score1 = sum(x1==x2 for x1,x2 in zip(prediction1,y_test))/len(y_test)
score2 = sum(x1==x2 for x1,x2 in zip(prediction2,y_data))/len(y_data)

RESULTADOS.loc['modelo_ID3'] = [score0, score1, score2]
RESULTADOS

In [ ]:
# WARNING: tarda varios minutos, se pueden reducir los parámetros para que tarde menos, o si se prefiere, 
# más abajo se detalla los parámetros óptimos obtenidos

parametersID3 = {
    'max_depth':[5, 10, 15],
    'min_samples_split': [1, 5, 10, 20],
    'prune' : [True, False],
    'gain_ratio' : [True, False],
    'min_entropy_decrease' : [0.0, 0.001, 0.01],
    'is_repeating' : [True, False]
}

modeloID3_opt = GridSearchCV(Id3Estimator(), parametersID3, cv = cross_validation, scoring='accuracy')
modeloID3_opt.fit(X_train,y_train)

Como puede observarse, los mejores parámetros para el árbol ID3 son:
<li> gain_ratio: True </li>
<li> is_repeating: False </li>
<li> max_depth: 10 </li>
<li> min_entropy_decrease: 0.0 </li>
<li> min_samples_split: 1 </li>
<li> prune: False </li>

In [ ]:
RESULTADOS.loc['modelo_ID3_opt'] = [modeloID3_opt.score(X_train,y_train), modeloID3_opt.score(X_test,y_test), modeloID3_opt.score(X_data,y_data)]
RESULTADOS

Como puede verse, este algoritmo no ofrece un cambio significativo respecto a CART (posiblemente se aprecien más en datasets más complejos). Podemos representar este nuevo árbol para estudiar si es similar a los generados por el otro algoritmo

In [ ]:
from id3 import export_graphviz #No confundir con el que estabamos usando antes 

export_graphviz(
    modeloID3.tree_,
    out_file = "modeloID3.dot",
    feature_names=dataset.columns)
with open("modeloID3.dot") as f:
    dot_grafo = f.read()
graphviz.Source(dot_grafo)

### 2.e) Matriz de confusión <a name="ej2-e"> </a>

Hasta ahora hemos analizado el rendimiento en función de la relación entre número de aciertos y total, sin embargo, en un dataset de diabetes como el que estamos tratando es importante conocer que porcentaje de los aciertos son falsos positivos, porque no es lo mismo equivocarse prediciendo que el paciente tiene diabetes cuando no lo tiene que al reves.<br>
Para ello vamos ha realizar una matriz de confusión con los datos del primer modelo de todos (podríamos hacerlo con cualquiera)

In [ ]:
y_pred = modelo.predict(X_data)

c_matrix = confusion_matrix(y_data, y_pred)
c_matrix

Esta matriz de confusión puede ser representada mediante un mapa de calor con seaborn:

In [ ]:
sns.heatmap(c_matrix, annot=True, fmt='d', cmap='Reds')

### 2.f) Otras métricas <a name="ej2-f"> </a>

otras métricas que podemos utilizar son la precisión, cobertura (_recall_), medida f1. Estas son métricas que dan diferente importancia al tipo de error (p.e. falsos positivos o falsos negativos en clasificación binaria). Pueden ser de utilidad para sistemas en los que nos preocupan más unos errores que otros.
Además también podemos calcular dos medidas que evaluan el ratio entre true positive y false positive, estos son: TPR (_True Positive Rate_) y FPR (_False Positive Rate_).

El significado intuitivo de las métricas es el siguiente:
- _precision_: grado de acierto en las instancias propuestas como positivas (¿son todos los que están?)
- _recall_: porcentaje de recuperación del total de las instancias positivas (¿están todos los que son?)
- _f1_ : media armónica de precisión y cobertura.
- _TPR_: positivos correctos (TP) dividido entre todos los positivos (TP+FN). También se denomina a esta métrica _sensibilidad_ y _cobertura_.
- _FPR_: positivos erróneos (FP) dividido entre todos los negativos (FP+TN). A la métrica $1-FPR$ también se le denomina _especificidad_.

Al combinar dos métricas complementarias, la medida _f1_ es apropiada para datasets que no estén bien balanceados.

In [ ]:
#True-negative (tn), false-positive(fp), false-negative(fn), true-positive(tp)
tn, fp, fn, tp = c_matrix.ravel()

precision = precision_score(y_data, y_pred, average='binary')
print("precision: ", precision)

recall = recall_score(y_data, y_pred, average='binary')
print("recall: ", recall)

f1 = f1_score(y_data, y_pred, average='binary')
print("f1: ", f1)

tpr = tp/(tp+fn)
print("TPR: ", tpr)

fpr = fp/(fp+tn)
print("fpr: ", fpr)

Además de estas métricas, tambien podemos hacer un análisis ROC con los ratios calculados, cabe destacar que en el análisis ROC cuanto mayor es el area bajo la curva mejor es el rendimiento de nuestro modelo

In [ ]:
fpr, tpr, threshold = roc_curve(y_data, y_pred)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

A simple vista puede observarse que el resultado es bastante bueno, pero podemos concretar la medida calculando el area bajo la curva:

In [ ]:
area = roc_auc_score(y_data, y_pred)
print("area bajo la curva:", area)